In [1]:
# Importa as bibliotecas necessárias

import csv
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from matplotlib.text import Annotation
import seaborn as sns

%matplotlib inline

In [2]:
# Abre o novo arquivo e cria um Dataframe
ver_rj = pd.read_csv("output/vereadores_2016_RJ.csv", encoding="latin-1")
vr = pd.DataFrame(ver_rj)

In [3]:
# Filtro para separar veradores eleitos e não eleitos.

# Listas que guardam as variáveis de cada tipo de situação eleitoral
var_eleitos = ['ELEITO POR MÉDIA', 'ELEITO POR QP']
var_nao_eleitos = ['NÃO ELEITO', 'SUPLENTE']

# Resultado booleano gerado analisando se os dados das listas estão
# ou não nas linhas do Dataframe
if_eleitos = vr.DS_SIT_TOT_TURNO.isin(var_eleitos)
if_nao_eleitos = vr.DS_SIT_TOT_TURNO.isin(var_nao_eleitos)

#Variáveis para os filtros no Dataframe
eleitos = vr[if_eleitos]
nao_eleitos = vr[if_nao_eleitos]

eleitos.head(1)

,NM_MUNICIPIO,NR_ZONA,DS_CARGO,NM_URNA_CANDIDATO,NR_PARTIDO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS
334,RIO DE JANEIRO,214,Vereador,TARCÍSIO MOTTA,50,PSOL,ELEITO POR QP,1594


In [9]:
el2 = eleitos.set_index('NM_URNA_CANDIDATO')
el2.loc['MARIELLE FRANCO']

,NM_MUNICIPIO,NR_ZONA,DS_CARGO,NR_PARTIDO,SG_PARTIDO,DS_SIT_TOT_TURNO,QT_VOTOS_NOMINAIS
NM_URNA_CANDIDATO,,,,,,,
MARIELLE FRANCO,RIO DE JANEIRO,170,Vereador,50,PSOL,ELEITO POR QP,1090
MARIELLE FRANCO,RIO DE JANEIRO,245,Vereador,50,PSOL,ELEITO POR QP,181
MARIELLE FRANCO,RIO DE JANEIRO,188,Vereador,50,PSOL,ELEITO POR QP,212
MARIELLE FRANCO,RIO DE JANEIRO,121,Vereador,50,PSOL,ELEITO POR QP,187
MARIELLE FRANCO,RIO DE JANEIRO,189,Vereador,50,PSOL,ELEITO POR QP,260
MARIELLE FRANCO,RIO DE JANEIRO,166,Vereador,50,PSOL,ELEITO POR QP,1170
MARIELLE FRANCO,RIO DE JANEIRO,161,Vereador,50,PSOL,ELEITO POR QP,1688
MARIELLE FRANCO,RIO DE JANEIRO,124,Vereador,50,PSOL,ELEITO POR QP,145
MARIELLE FRANCO,RIO DE JANEIRO,14,Vereador,50,PSOL,ELEITO POR QP,253


In [ ]:
# Filtro partido por Zona
voto_zona = eleitos.SG_PARTIDO.isin(['PSOL'])
vz_total = eleitos[voto_zona]

benfica = vz_total[vz_total['NR_ZONA'] == 193]
benfica

In [ ]:
candidato = eleitos.NM_URNA_CANDIDATO.isin(['MARIELLE FRANCO'])
cand = eleitos[candidato]

#vz_total[vz_total['QT_VOTOS_NOMINAIS'] > 2000]


plt.scatter(cand.NR_ZONA, cand.QT_VOTOS_NOMINAIS, color='b')

In [ ]:
# Votos por Zona - eleitos
plt.scatter(eleitos.QT_VOTOS_NOMINAIS, eleitos.NR_ZONA,
            c='#06975e', edgecolors='#FFFFFF')
plt.xlabel('Votos')
plt.ylabel('Zonas Eleitorais')
plt.title('Votos por Zona - Eleitos')

In [ ]:
# Votos por Zona - não eleitos
plt.scatter(nao_eleitos.QT_VOTOS_NOMINAIS, nao_eleitos.NR_ZONA)
plt.xlabel('Votos')
plt.ylabel('Zonas Eleitorais')
plt.title('Votos por Zona - Geral')

vr.count()